# AlexNet

In [1]:
import time
import torch
import torchvision
from torch import nn, optim
import renyan_utils as ry
from torchsummary import summary

In [2]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.conv = nn.Sequential(nn.Conv2d(1, 96, 11, 4, 3),
                                 nn.ReLU(),
                                 nn.MaxPool2d(3, 2),
                                 nn.Conv2d(96, 256, 5, 1, 2),
                                 nn.ReLU(),
                                 nn.MaxPool2d(3, 2),
                                 nn.Conv2d(256, 384, 3, 1, 1),
                                 nn.ReLU(),
                                 nn.Conv2d(384, 384, 3, 1, 1),
                                 nn.ReLU(),
                                 nn.Conv2d(384, 256, 3, 1, 1),
                                 nn.ReLU(),
                                 nn.MaxPool2d(3, 2))
        self.fc = nn.Sequential(nn.Linear(256*6*6, 4096),
                               nn.ReLU(),
                               nn.Dropout(0.5),
                               nn.Linear(4096, 4096),
                               nn.ReLU(),
                               nn.Dropout(0.5),
                               nn.Linear(4096, 10))
    
    def forward(self, img):
        feature = self.conv(img)
        output = self.fc(feature.view(img.shape[0], -1))
        return output

In [3]:
net = AlexNet()
# net

In [4]:
summary(net, (1, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 55, 55]          11,712
              ReLU-2           [-1, 96, 55, 55]               0
         MaxPool2d-3           [-1, 96, 27, 27]               0
            Conv2d-4          [-1, 256, 27, 27]         614,656
              ReLU-5          [-1, 256, 27, 27]               0
         MaxPool2d-6          [-1, 256, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         885,120
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 384, 13, 13]       1,327,488
             ReLU-10          [-1, 384, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         884,992
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
           Linear-14                 [-

## Data

In [5]:
def load_data_fashion_mnist_resize(batch_size, resize = None, root = './data/'):
    trans = []
    if resize:
        trans.append(torchvision.transforms.Resize(size = resize))
    trans.append(torchvision.transforms.ToTensor())
    
    transform = torchvision.transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(root = root, train = True, download = False, transform = transform)
    mnist_test = torchvision.datasets.FashionMNIST(root = root, train = False, download = False, transform = transform)
    train_iter = torch.utils.data.DataLoader(mnist_train, batch_size = batch_size, shuffle = True)
    test_iter = torch.utils.data.DataLoader(mnist_test, batch_size = batch_size, shuffle = False)
    
    return train_iter, test_iter

In [6]:
batch_size = 32
train_iter, test_iter = load_data_fashion_mnist_resize(batch_size, resize = 224)

In [7]:
X, y = next(iter(test_iter))

In [8]:
X.shape

torch.Size([32, 1, 224, 224])

In [13]:
lr, num_epochs = 0.001, 5
optimizer = torch.optim.Adam(net.parameters(), lr = lr)
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
ry.train_mnist_net(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)